Compute VAEP Values & Calculate Player Ratings

In [2]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [216]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

from config import chain_file_path
from vaep.domain.contracts.modelling_data_contract import ModellingDataContract
from vaep.domain.modelling.supermodel import SuperXGBClassifier
from vaep.domain.modelling.model_evaluation import XGBClassifierEvaluator
from vaep.domain.preprocessing.preprocessing import *
from vaep.domain.preprocessing.formula import *


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
FEATURES = ModellingDataContract.feature_list_scores

Load chain data

In [4]:
chains = pd.read_csv(chain_file_path)
chains.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
1169201,252,ballUp,turnover,1984.0,4,1937,1942.0,St Kilda,St Kilda,Dan Butler,Dan_Butler,Kick,-65.0,14.0,clanger,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169202,252,ballUp,turnover,1985.0,4,1937,1945.0,St Kilda,St Kilda,Dan Butler,Dan_Butler,Out On Full After Kick,-69.0,34.0,NaN,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169203,253,possGain,behind,1986.0,4,1949,1949.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,OOF Kick In,67.0,-34.0,NaN,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169204,253,possGain,behind,1987.0,4,1949,1976.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,Kick,67.0,-34.0,ineffective,True,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169205,253,possGain,behind,1988.0,4,1949,1978.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,Behind,67.0,-34.0,NaN,NaN,missLeft,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0


Preprocess

In [5]:
schema_chains = convert_chains_to_schema(chains)
gamestate_features = create_gamestate_features(schema_chains)

In [6]:
vaep_modelling_data = pd.concat([schema_chains, gamestate_features], axis=1)

In [12]:
vaep_features = vaep_modelling_data[FEATURES]

Load models

In [7]:
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/model_outputs/models/"
scoring_model = "vaep_scoring_v1.joblib"
conceding_model = "vaep_conceding_v1.joblib"

In [9]:
score_model = joblib.load(model_file_path + "/" + scoring_model)
concede_model = joblib.load(model_file_path + "/" + conceding_model)

Make predictions

In [14]:
schema_chains['scores'] = score_model.predict_proba(vaep_features, calibrate=True)
schema_chains['concedes'] = concede_model.predict_proba(vaep_features, calibrate=True)

Compute VAEP

In [147]:
match_list = list(schema_chains['match_id'].unique())
match_vaep_list = []
for match in match_list:
    match_chains = schema_chains[schema_chains['match_id'] == match]
    v = value(match_chains, match_chains['scores'], match_chains['concedes'])
    match_vaep_list.append(v)
    
vaep_values = pd.concat(match_vaep_list, axis=0)

In [148]:
chain_vaep_data = pd.concat([schema_chains, vaep_values], axis=1)
chain_vaep_data.head()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value
1,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.052962,0.009920,0.000000,-0.000000,0.000000
2,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.022556,0.050321,-0.030406,-0.040401,-0.070807
3,202101_BrisbaneLions_Sydney,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.074422,0.008536,0.024101,0.014020,0.038121
4,202101_BrisbaneLions_Sydney,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.084299,0.013648,0.009877,-0.005111,0.004766
5,202101_BrisbaneLions_Sydney,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.215944,0.012438,0.131645,0.001209,0.132854


In [261]:
chain_vaep_data.to_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/schema_chains_vaep_values.csv", index=False)